# Qwen 0.5b en GRPO
Entrenamiento de un modelo pequeño para razonamiento matemático con aprendizaje por refuerzo

En este notebook, exploramos el uso de Qwen-0.5B junto con [GRPO demo](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) para entrenar un modelo en tareas de razonamiento matemático. Utilizaremos el conjunto de datos GSM8K, un benchmark diseñado para evaluar la capacidad de los modelos en la resolución de problemas matemáticos de nivel escolar.

Además, aprovecharemos vLLM para la generación de texto, lo que permite una ejecución más eficiente en términos de velocidad y uso de recursos.

### Configuración del entorno
Como primer paso, instalaremos vLLM. Es importante tener en cuenta que, tras la instalación, será necesario reiniciar la sesión para aplicar los cambios correctamente.

## 1.Instalación de dependencias: vllm

En esta primera celda se instala la librería vllm. Esta librería acelera la generación de textos, lo que resulta importante para entrenar el modelo de forma más eficiente.
Nota: Al finalizar la instalación, es posible que se deba reiniciar la sesión de Colab.

In [ ]:
!pip install vllm

## 2. Instalación de trl y datasets
En esta celda se instalan las librerías trl (para entrenamiento con RL) y datasets (para cargar y procesar datasets).
Importante: Se instala trl y datasets en este orden, ya que instalar vllm después puede generar algún bug en trl.

In [ ]:
!pip install trl datasets

## 3. Definición del prompt y carga del dataset

### 3.1. Definir la estructura general del prompt
Se define la estructura del prompt que se utilizará para la interacción. El formato consiste en dos secciones:

reasoning ... /reasoning: Donde se espera el razonamiento del modelo.

answer ... /answer: Donde se espera la respuesta final.

In [ ]:
# [3] Definir el prompt y cargar librerías necesarias
import re
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import GRPOConfig, GRPOTrainer

# Prompt de sistema que indica el formato esperado en la respuesta
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

# Formato XML para el chain-of-thought (razonamiento en cadena)
XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""


INFO 02-08 12:49:01 __init__.py:190] Automatically detected platform cuda.


### 3.2. Preparación y reestructuración del dataset GSM8K
El dataset gsm8k es importado desde Hugging Face. Se define una función para extraer la respuesta en formato XML o en formato con “####” (esto último se utiliza en la función de extracción de respuesta) y se crea una función que transforma cada ejemplo del dataset para que se ajuste a un formato de conversación. El prompt incluye un mensaje de sistema (con el formato esperado) y el mensaje del usuario con la pregunta.

In [ ]:
# [4] Funciones para extraer la respuesta y cargar el dataset GSM8K

def extract_xml_answer(text: str) -> str:
    """
    Extrae el contenido entre las etiquetas <answer> y </answer>.
    """
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    """
    Extrae la respuesta cuando se encuentra separada por '####'.
    Si no se encuentra '####', devuelve None.
    """
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# Función para obtener las preguntas del dataset gsm8k y estructurarlas como una conversación
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split]  # Carga del dataset (usar split 'train' o 'test')
    data = data.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    })
    return data

# Cargamos el dataset transformado
dataset = get_gsm8k_questions()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

## 4. Definición de funciones de recompensa

En esta sección se definen las funciones de recompensa que se utilizarán para evaluar las respuestas generadas por el modelo durante el entrenamiento. Estas funciones evalúan aspectos como la correctitud de la respuesta, el formato de la respuesta (estricto y suave), el formato interno XML y un recompensa basada en la validez numérica.

### 4.1. Función de recompensa por correctitud
Esta función compara la respuesta extraída del razonamiento generado por el modelo con la respuesta correcta (del dataset).
Si coinciden, se asigna una recompensa de 2.0; en caso contrario, 0.0.

In [ ]:
# [5] Funciones de recompensa

def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    # Se extrae el contenido generado por el modelo
    responses = [completion[0]['content'] for completion in completions]
    # Se extrae la pregunta del prompt
    q = prompts[0][-1]['content']
    # Se extrae la respuesta dentro de las etiquetas <answer>
    extracted_responses = [extract_xml_answer(r) for r in responses]
    # Se imprime la pregunta, la respuesta correcta, la respuesta generada y la extraída
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    # Devuelve 2.0 si la respuesta extraída coincide con la respuesta correcta, sino 0.0
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]


### 4.2. Recompensa por respuesta numérica
Verifica si la respuesta extraída es un dígito. Si es así, se le asigna una recompensa de 0.5.

In [ ]:
def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]


### 4.3. Recompensa por formato estricto
Comprueba si la respuesta generada respeta exactamente el siguiente formato:

In [ ]:
'''
<reasoning>
...
</reasoning>
<answer>
...
</answer>
'''

'\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n'

Si coincide, se otorga 0.5 de recompensa.

In [ ]:
def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Función de recompensa que comprueba que la respuesta tenga un formato específico."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]


### 4.4. Recompensa por formato suave
Esta función es similar a la anterior, pero utiliza un patrón menos estricto para evaluar el formato de la respuesta.

In [ ]:
def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Función de recompensa que comprueba si la respuesta tiene el formato esperado de manera flexible."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]


###4.5. Recompensa basada en el conteo de etiquetas XML
Esta función analiza el texto generado y otorga pequeños incrementos de recompensa en función de la presencia y posición de las etiquetas reasoning, /reasoning, answer y /answer. Además, penaliza ligeramente si hay caracteres extra al final.

In [ ]:
def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]


## 5. Configuración de los argumentos de entrenamiento y preparación del modelo

En esta sección se definen los parámetros de entrenamiento y se carga el modelo y el tokenizador.

### 5.1. Definición de parámetros de entrenamiento
Se utilizan las configuraciones de GRPO (Generalized Reward Policy Optimization) para especificar los hiperparámetros del entrenamiento, tales como tasa de aprendizaje, regularización, tamaño de batch, número de épocas, etc. También se especifica el uso de vllm y se indica cuánta memoria de la GPU se debe utilizar.

In [ ]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

output_dir="outputs/Qwen-0.5B-GRPO"
run_name="Qwen-0.5B-GRPO-gsm8k"

training_args = GRPOConfig(
    output_dir=output_dir,
    run_name=run_name,
    learning_rate=5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type='cosine',
    logging_steps=1,
    fp16=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_generations=16,
    max_prompt_length=256,
    max_completion_length=200,
    num_train_epochs=1,
    save_steps=100,
    max_grad_norm=0.1,
    log_on_each_node=False,
    use_vllm=True,
    vllm_gpu_memory_utilization=.3,
    vllm_device="cuda:0",
    report_to="none" ## Deshabilitamos el reporte a Wandb.
)


### 5.2. Carga del modelo y tokenizador
Se carga el modelo preentrenado Qwen/Qwen2.5-0.5B-Instruct en formato fp16 (media precisión) y se mueve a la GPU. Además, se carga el tokenizador correspondiente y se asigna el token de padding al token de fin de secuencia (EOS).

In [ ]:
# Carga del modelo y del tokenizador
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map=None
).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


## 6. Verificación de la disponibilidad de la GPU

Se ejecuta una pequeña celda para verificar que la GPU esté disponible y para identificarla.

In [ ]:
# [7] Verificar la disponibilidad de la GPU
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))


True
1
Tesla T4


## 7. Ejecución del entrenamiento
Finalmente, se lanza el entrenamiento usando el GRPOTrainer. Se define nuevamente (con ligeros cambios) la configuración de entrenamiento, se carga el modelo y el tokenizador, se configura una variable de entorno para que vllm use float16, y se instancian las funciones de recompensa que se utilizarán durante el entrenamiento.
En la celda final se invoca el método trainer.train() para iniciar el proceso de entrenamiento.

In [ ]:
# [8] Lanzamiento del entrenamiento

model_name = "Qwen/Qwen2.5-0.5B-Instruct"
output_dir = "outputs/Qwen-0.5B-GRPO"
run_name = "Qwen-0.5B-GRPO-gsm8k"

training_args = GRPOConfig(
    output_dir=output_dir,
    run_name=run_name,
    optim="adamw_torch_fused",  ######
    learning_rate=5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.2,  # Mayor regularización L2
    warmup_ratio = 0.1,
    lr_scheduler_type='cosine',
    logging_steps=1,
    fp16=True,  ####
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_generations=8,
    max_prompt_length=128,
    max_completion_length=100,
    num_train_epochs=1,
    save_steps=100,
    max_grad_norm=0,  # Deshabilita el clipping de gradiente
    log_on_each_node=False,
    use_vllm=False,  # En esta configuración se deshabilita vllm (se puede cambiar según se desee)
    vllm_gpu_memory_utilization=0.3,
    vllm_device="cuda:0",
    report_to="none"  # Se deshabilita el reporte a Wandb.
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map=None  # Nota: se pasa None para no usar device_map automático.
).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

import os
os.environ["VLLM_DTYPE"] = "float16"  # Configura vllm para usar media precisión

# Instanciamos el trainer con el modelo, el tokenizador (processing_class) y las funciones de recompensa.
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func
    ],
    args=training_args,
    train_dataset=dataset,
    # peft_config=peft_config  # (comentado: si se utiliza PEFT se configuraría aquí)
)

# Inicia el entrenamiento
trainer.train()


-------------------- Question:
Ahmed and Emily are having a contest to see who can get the best grade in the class. There have been 9 assignments and Ahmed has a 91 in the class. Emily has a 92. The final assignment is worth the same amount as all the other assignments. Emily got a 90 on the final assignment. What is the minimum grade Ahmed needs to get to beat Emily if all grades are whole numbers? 
Answer:
100 
Response:
To determine the minimum grade Ahmed needs to beat Emily, we can follow these steps:

1. Identify the total possible grades for the final assignment: 91 (the score of Ahmed) + 92 (the score of Emily) = 183 possible scores.
2. The final assignment is worth the same amount as all other assignments, so it is a total of 183 possible scores.
3. Ahmed has already achieved a score of 91.
4. Emily 
Extracted:
To determine the minimum grade Ahmed needs to beat Emily, we can follow these steps:

1. Identify the total possible grades for the final assignment: 91 (the score of A

Step,Training Loss
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000
10,0.000000


Se han truncado las últimas 5000 líneas del flujo de salida.
   So, Debelyn has 18 dolls after the gift.

2. **Calculate the number of dolls Christel has
-------------------- Question:
Henry, John and Leo collected some seashells from the beach. Henry collected 11, Paul 24. If they initially collected 59 seashells in total and Leo gave a quarter of his collection to a younger kid they met, how many do they have in total now? 
Answer:
53 
Response:
Henry collected 11 * 100 = 1100 new seashells. After that, the total number of seashells remained at 59 + 1100 = 1159. Out of them, 240 / 4 = 60 seashells were given to the kid. Consequently, the new total number of seashells was 1159 - 60 = 1159. 
Extracted:
Henry collected 11 * 100 = 1100 new seashells. After that, the total number of seashells remained at 59 + 1100 = 1159. Out of them, 240 / 4 = 60 seashells were given to the kid. Consequently, the new total number of seashells was 1159 - 60 = 1159.
-------------------- Question:
Delta, Be

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
